# Junior Data Scientist Technical Assignment
# Andrew McEllistrim

## Importing all Neccesary packages


In [1]:



import numpy as np 
import pandas as pd 
import geopandas as gpd

from shapely.geometry import Point
from shapely.geometry import LineString
from shapely.ops import nearest_points

import folium
from folium import Marker

## Set up script  based on assignment and plot a map from Folium
Upon first plotting map we see the points as defined correspond to a location in the Indian Ocean. 
Thus we assume the Longitude and Latitude are reversed. Upon testing this we recover the map shown in the assignment. Here we use np.flip() to correct the points

In [2]:
points=[(-0.1421021, 51.5410616),(-0.1422064, 51.5410683),(-0.1423133, 51.5410717),
        (-0.1423844, 51.5410881),(-0.1425594, 51.5410901),(-0.1427707, 51.5410819),
        (-0.1429775, 51.5410709),(-0.1431874, 51.54105),(-0.1433833, 51.5410192),
        (-0.1437212, 51.5409178),(-0.1440496, 51.5407818),(-0.1440836, 51.5407693),
        (-0.1442231, 51.5407181)]
points=np.flip(points,axis=1)

#Draw a map to visualise problem
map = folium.Map(location=points[6], tiles='CartoDB Positron', zoom_start=18,
                 zoom_control=False,scrollWheelZoom=False,dragging=False)
folium.PolyLine(points, color="red", weight=2.5, opacity=1).add_to(map)
map

## Create our GeoPanda Dataframes

In [3]:
# Define the WGS line string geometry
wgs_geometry = LineString(points)

# Define the LRS line string geometry
lrs_geometry = LineString([(0, 0),(157.36859375593266, 0)])

# Convert WGS geometry to GeoDataFrame
wgs_gdf = gpd.GeoDataFrame(geometry=[wgs_geometry],crs="EPSG:4326")

# Convert LRS geometry to GeoDataFrame
lrs_gdf = gpd.GeoDataFrame(geometry=[lrs_geometry])

## Task 1
Convert points from lrs to wgs 

Point 1 = (0,0) 

Point 2 = (106, 0) 

In [4]:
#define points
lrs_point_1 = Point(0, 0)
lrs_point_2 = Point(106, 0)

#create function
def LRS_to_WGS(LRS_PT,wgs_gdf,lrs_gdf):
    LRS_PT=np.asarray(LRS_PT.coords)[0, 0] 
    LRS_LINESTRING = lrs_gdf['geometry'].iloc[0]      #extracts the linestring from the lrs_gdf
    TOTAL_LRS_VAL=np.asarray(LRS_LINESTRING.coords)[-1,0] 
    SCALE=LRS_PT/TOTAL_LRS_VAL 
    WGS_PT=wgs_gdf['geometry'].iloc[0].interpolate(SCALE,1)
    WGS_PT = Point(round(WGS_PT.x, 7), round(WGS_PT.y, 7))
    return WGS_PT

#Get answers and then print results
wgs_result_1 = LRS_to_WGS(lrs_point_1,wgs_gdf,lrs_gdf)
wgs_result_2 = LRS_to_WGS(lrs_point_2,wgs_gdf,lrs_gdf)

print("Part One:")
print("POINT(0 0) in WGS:", wgs_result_1)
print("POINT(106 0) in WGS:", wgs_result_2)

Part One:
POINT(0 0) in WGS: POINT (51.5410616 -0.1421021)
POINT(106 0) in WGS: POINT (51.5409671 -0.143557)


## Task 2
Convert points from wgs to lrs (note points flipped to match convention as defined earlier)

Point 1=(51.5410819 -0.1427707)

Point 2=(51.5410498 -0.1431889)

In [5]:
#define points
wgs_point_1 = Point(51.5410819, -0.1427707)
wgs_point_2 = Point(51.5410498, -0.1431889)

#create function
def wgs_to_lrs(WGS_PT,wgs_gdf,lrs_gdf):
    LIN_REF = wgs_gdf['geometry'].iloc[0].project(WGS_PT) #needs scaling as length in WGS non-intuitive
    LRS_LINESTRING = lrs_gdf['geometry'].iloc[0]
    END=np.asarray(LRS_LINESTRING.coords)[-1,0] 
    SCALED_LIN_REF=LIN_REF/wgs_gdf['geometry'].iloc[0].length * END #scaling answer to LRS scale
    SLF=round(SCALED_LIN_REF,2)
    SLF=Point(SLF,0) #returns output to point format
    return SLF

#Get answers and then print results
lrs_result_1 = wgs_to_lrs(wgs_point_1,wgs_gdf,lrs_gdf)
lrs_result_2 = wgs_to_lrs(wgs_point_2,wgs_gdf,lrs_gdf)

print("Part Two:")
print("POINT(51.5410819 -0.1427707) in LRS:", lrs_result_1)
print("POINT(51.5410498 -0.1431889) in LRS:", lrs_result_2)

Part Two:
POINT(51.5410819 -0.1427707) in LRS: POINT (48.42 0)
POINT(51.5410498 -0.1431889) in LRS: POINT (78.71 0)


## Task 3
**‘snap’** the following points to the nearest bit of the road carriageway and provide both the wgs_geometry and lrs_geometry of the snapped point:

Point 1= (51.5411983 -0.1426069)

Point 2= (51.5404112 -0.1434099)

Point 3= (51.5412199 -0.1478768)


In [6]:
#define points
snap_point_1=Point(51.5411983,-0.1426069, )
snap_point_2=Point(51.5404112, -0.1434099, )
snap_point_3=Point(51.5412199, -0.1478768, )

#create function
def snap_to_nearest(SNAP_PT,wgs_gdf,lrs_gdf):
    LINE=wgs_gdf['geometry'].iloc[0]
    NPOL, _ = nearest_points(LINE, SNAP_PT) #NPOL= nearest point on line
    NPOL = Point(round(NPOL.x, 7), round(NPOL.y, 7))
    NPOL_LRS=wgs_to_lrs(NPOL,wgs_gdf,lrs_gdf)
    return NPOL, NPOL_LRS

#Get answers and then print results
NPOL1,NPOL_LRS1=snap_to_nearest(snap_point_1,wgs_gdf,lrs_gdf)
NPOL2,NPOL_LRS2=snap_to_nearest(snap_point_2,wgs_gdf,lrs_gdf)
NPOL3,NPOL_LRS3=snap_to_nearest(snap_point_3,wgs_gdf,lrs_gdf)

print('Part 3:')
print('Given',snap_point_1,', this snaps onto the line at',NPOL1,'in WGS and',NPOL_LRS1,'in LRS')
print('Given',snap_point_2,', this snaps onto the line at',NPOL2,'in WGS and',NPOL_LRS2,'in LRS')
print('Given',snap_point_3,', this snaps onto the line at',NPOL3,'in WGS and',NPOL_LRS3,'in LRS')


Part 3:
Given POINT (51.5411983 -0.1426069) , this snaps onto the line at POINT (51.5410884 -0.1426026) in WGS and POINT (36.28 0) in LRS
Given POINT (51.5404112 -0.1434099) , this snaps onto the line at POINT (51.5409616 -0.1435751) in WGS and POINT (107.36 0) in LRS
Given POINT (51.5412199 -0.1478768) , this snaps onto the line at POINT (51.5407181 -0.1442231) in WGS and POINT (157.37 0) in LRS
